# Avaliação de padrões problemáticos em voos de janeiro de 2023

O objetivo deste estudo é analisar padrões de atrasos, cancelamentos e desvios em voos domésticos, identificando segmentos que diferenciam o comportamento operacional das companhias aéreas, aeroportos e períodos de tempo. A partir da aplicação de estatísticas descritivas, segmentações, testes de hipótese, cálculo de risco relativo e modelos de regressão, busca-se compreender não apenas se existem diferenças significativas entre grupos, mas também a magnitude e direção dessas diferenças. O propósito final é gerar insights acionáveis que apoiem decisões estratégicas para melhorar a eficiência operacional, otimizar recursos e reduzir impactos para passageiros e companhias.

Acesse as sessões posterior ao **Gerencimento do notebook** para conferir o dashboard

### Gerenciamento do notebook

#### Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from math import atan2, degrees
import warnings
warnings.filterwarnings('ignore')

import dash
from dash import html, dcc, Input, Output, callback
import dash_bootstrap_components as dbc

#### Definições

In [2]:
df = pd.read_csv("project_development/dataset/created/df_view.csv")

In [3]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

In [4]:
# Aplicar ordenação categórica
weekday_order = ["Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sábado", "Domingo"]
period_order = ["Madrugada", "Manhã", "Tarde", "Noite"]

df["DAY_OF_WEEK"] = pd.Categorical(df["DAY_OF_WEEK"], categories=weekday_order, ordered=True)
df["TIME_PERIOD"] = pd.Categorical(df["TIME_PERIOD"], categories=period_order, ordered=True)

In [5]:
print(f"Dataset carregado com sucesso: {len(df)} registros")
print(df.columns)

Dataset carregado com sucesso: 536720 registros
Index(['Unnamed: 0', 'FL_DATE', 'FL_DAY', 'ORIGIN_CITY', 'ORIGIN_STATE',
       'DEST_CITY', 'CANCELLED', 'DIVERTED', 'DELAY', 'DISTANCE',
       'AIRLINE_Description', 'DELAY_OVERALL', 'TIME_PERIOD', 'DAY_OF_WEEK',
       'TIME_HOUR', 'ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON'],
      dtype='object')


#### Funções de cálculo e processamento

In [6]:
def calculate_big_numbers(df):
    """Calcula as métricas principais"""
    total_flights = len(df)
    avg_delay = df['DELAY_OVERALL'].mean()
    delay_percentage = (df['DELAY'].sum() / total_flights) * 100
    cancelled_percentage = (df['CANCELLED'].sum() / total_flights) * 100
    diverted_percentage = (df['DIVERTED'].sum() / total_flights) * 100
    
    return {
        'total_flights': total_flights,
        'avg_delay': avg_delay,
        'delay_percentage': delay_percentage,
        'cancelled_percentage': cancelled_percentage,
        'diverted_percentage': diverted_percentage
    }

In [7]:
def create_metric_data(df, group_col, metric, observed=True):
    """Cria dados agrupados para métricas específicas"""
    if metric == 'avg_delay':
        data = df.groupby(group_col, observed=observed)['DELAY_OVERALL'].mean().sort_values(ascending=False)
        title_suffix = "Atraso Médio (min)"
    elif metric == 'delay_count':
        data = df.groupby(group_col, observed=observed)['DELAY'].sum().sort_values(ascending=False)
        title_suffix = "Quantidade de Atrasos"
    elif metric == 'cancelled_count':
        data = df.groupby(group_col, observed=observed)['CANCELLED'].sum().sort_values(ascending=False)
        title_suffix = "Quantidade de Cancelamentos"
    elif metric == 'diverted_count':
        data = df.groupby(group_col, observed=observed)['DIVERTED'].sum().sort_values(ascending=False)
        title_suffix = "Quantidade de Desvios"
    else:
        data = df.groupby(group_col, observed=observed)['DELAY_OVERALL'].mean().sort_values(ascending=False)
        title_suffix = "Atraso Médio (min)"
    
    return data, title_suffix

#### Funções de visualização

In [8]:
def create_simple_bar_chart(data, title, x_label, y_label):
    """Cria um gráfico de barras simplificado"""
    if len(data) == 0:
        fig = px.bar(title=f"{title} (Sem dados)")
        fig.update_layout(height=400, title_x=0.5)
        return fig
         
    y_values = [str(x) for x in data.index]
    
    fig = px.bar(
        x=data.values,
        y=y_values,
        orientation="h",
        title=f"<b>{title}</b>",
        labels={"x": x_label, "y": y_label},
        color=data.values,
        color_continuous_scale="deep"
    )
    
    fig.update_layout(
        height=400,
        yaxis={"categoryorder": "total ascending"},
        title_x=0.5,
        title_font_size=14,
        font=dict(size=10),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=50, r=20, t=60, b=40),
        showlegend=False,
        coloraxis_showscale=False
    )
    return fig

In [9]:
def create_line_chart_continuous(data, title, x_label, y_label, group_col):
    """Cria um gráfico de linhas contínuo com agrupamento correto"""
    if len(data) == 0:
        fig = px.line(title=f"{title} (Sem dados)")
        fig.update_layout(height=400, title_x=0.5)
        return fig
    
    if group_col in ['FL_DAY', 'TIME_HOUR']:
        data = data.sort_index()
        x_values = data.index
    else:
        x_values = data.index
    
    fig = px.line(
        x=x_values,
        y=data.values,
        title=f"<b>{title}</b>",
        labels={"x": x_label, "y": y_label}
    )
    
    fig.update_traces(
        line_color="#2166AC", 
        line_width=3, 
        marker=dict(size=6),
        mode="lines+markers"
    )
    
    fig.update_layout(
        height=400,
        title_x=0.5,
        title_font_size=14,
        font=dict(size=10),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=50, r=20, t=60, b=40),
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='lightgray'),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='lightgray')
    )
    return fig

In [10]:
def criar_mapa_rotas_avancado(df, top_n=30, altura=500, selected_metric='avg_delay'):
    """Cria um mapa interativo das rotas de voo"""
    
    metric_config = {
        'avg_delay': {'col': 'DELAY_OVERALL', 'agg': 'mean', 'title': 'Atraso Médio', 'unit': 'min'},
        'delay_count': {'col': 'DELAY', 'agg': 'sum', 'title': 'Quantidade de Atrasos', 'unit': 'voos'},
        'cancelled_count': {'col': 'CANCELLED', 'agg': 'sum', 'title': 'Quantidade de Cancelamentos', 'unit': 'voos'},
        'diverted_count': {'col': 'DIVERTED', 'agg': 'sum', 'title': 'Quantidade de Desvios', 'unit': 'voos'}
    }
    
    config = metric_config[selected_metric]
    
    # Verificar se as colunas de coordenadas existem
    required_cols = ['ORIGIN_CITY', 'DEST_CITY', 'ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        fig = go.Figure()
        fig.update_layout(
            title=f"Dados de coordenadas não disponíveis: {', '.join(missing_cols)}",
            height=altura
        )
        return fig

    # Agrupar por rota 
    rotas_data = (
        df.groupby(["ORIGIN_CITY", "DEST_CITY", "ORIGIN_LAT", "ORIGIN_LON", "DEST_LAT", "DEST_LON"], 
                           as_index=False)
        .agg({
            config['col']: config['agg'], 
            'FL_DATE': 'count',
            'TIME_HOUR': 'mean'  # Hora média para definir cor
        })
        .rename(columns={'FL_DATE': 'TOTAL_VOOS'})
        .sort_values(by=config['col'], ascending=False)
        .head(top_n)
        .dropna(subset=['ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON'])
    )
    
    if rotas_data.empty:
        fig = go.Figure()
        fig.update_layout(title="Nenhuma rota válida encontrada", height=altura)
        return fig
    
    fig = go.Figure()

    # Normalizar valores para espessura da linha (baseado na métrica)
    min_metric = rotas_data[config['col']].min()
    max_metric = rotas_data[config['col']].max()
    
    # Adicionar linhas das rotas com cor baseada em horário e espessura baseada na métrica
    for _, rota in rotas_data.iterrows():
        # COR MELHORADA: Escala HSL para azuis mais contrastantes
        hora_normalizada = rota['TIME_HOUR'] / 23
        
        # Hue: 200-220 (azul), Saturation: 70-100%, Lightness: 70-20% (claro → escuro)
        hue = 200 + (hora_normalizada * 20)        # 200 a 220
        saturation = 70 + (hora_normalizada * 30)  # 70% a 100%
        lightness = 70 - (hora_normalizada * 50)   # 70% a 20% (mais contraste)
        
        cor_hsl = f'hsl({hue}, {saturation}%, {lightness}%)'
        
        if max_metric > min_metric:
            # Normalizar a métrica entre 0 e 1
            metric_normalizada = (rota[config['col']] - min_metric) / (max_metric - min_metric)
            # Escala exponencial para melhor visualização (valores pequenos ficam mais visíveis)
            espessura = 1 + (metric_normalizada ** 0.7) * 9  # 1 a 10 (mais suave)
        else:
            espessura = 3
        
        # Garantir espessura mínima e máxima
        espessura = max(1, min(espessura, 10))
        
        fig.add_trace(go.Scattergeo(
            lon=[rota['ORIGIN_LON'], rota['DEST_LON']],
            lat=[rota['ORIGIN_LAT'], rota['DEST_LAT']],
            mode='lines',
            line=dict(width=espessura, color=cor_hsl),
            name=f"{rota['ORIGIN_CITY']} → {rota['DEST_CITY']}",
            showlegend=False,
            hovertemplate=(
                f"<b>%{{text}}</b><br>"
                f"{config['title']}: {rota[config['col']]:.1f} {config['unit']}<br>"
                f"Hora Média: {rota['TIME_HOUR']:.1f}h<br>"
                f"Total de Voos: {rota['TOTAL_VOOS']}<br>"
                f"Espessura: {espessura:.1f}<br>"
                "<extra></extra>"
            ),
            text=f"{rota['ORIGIN_CITY']} → {rota['DEST_CITY']}"
        ))
    
    # Marcadores de origem (verde)
    fig.add_trace(go.Scattergeo(
        lon=rotas_data['ORIGIN_LON'],
        lat=rotas_data['ORIGIN_LAT'],
        mode='markers',
        marker=dict(size=8, color='green', symbol='circle', line=dict(width=1, color='white')),
        text=rotas_data['ORIGIN_CITY'],
        name="Origem",
        hovertemplate="<b>%{text}</b><br><i>Aeroporto de Origem</i><extra></extra>"
    ))
    
    # Marcadores de destino (vermelho)
    fig.add_trace(go.Scattergeo(
        lon=rotas_data['DEST_LON'],
        lat=rotas_data['DEST_LAT'],
        mode='markers',
        marker=dict(size=8, color='red', symbol='circle', line=dict(width=1, color='white')),
        text=rotas_data['DEST_CITY'],
        name="Destino",
        hovertemplate="<b>%{text}</b><br><i>Aeroporto de Destino</i><extra></extra>"
    ))
    
    # Definir subtitle baseado na métrica
    subtitle_text = (
        f"🎨 Azul claro = madrugada ⭢ Azul escuro = noite | "
        f"📏 Espessura: atraso médio | "
        f"🟢 Origem | 🔴 Destino"
    )
    
    fig.update_layout(
        title=dict(
            text=f"<b>As {top_n} rotas com maior atraso</b><br>"
                 f"<sub>{subtitle_text}</sub>",
            x=0.5,
            xanchor='center'
        ),
        geo=dict(
            scope='usa',
            projection_type='albers usa',
            showland=True,
            landcolor='rgb(245, 245, 240)',
            showlakes=True,
            lakecolor='rgb(255, 255, 255)',
            showsubunits=True,
            subunitcolor='rgb(217, 217, 217)',
        ),
        height=altura,
        margin=dict(l=0, r=0, t=80, b=0),
        legend=dict(
            x=0.02,
            y=0.98,
            bgcolor='rgba(255, 255, 255, 0.9)',
            bordercolor='rgba(0, 0, 0, 0.2)',
            borderwidth=1
        ),
        showlegend=False,        
    )
    
    return fig

#### Componentes da interface

In [11]:
def create_number_card(icon, value, label, color):
    return dbc.Card([
        dbc.CardBody([
            html.H3(icon, className="text-center mb-2", style={'fontSize': '2em'}),
            html.H3(value, className="text-center mb-2", style={'fontSize': '2em', 'fontWeight': 'bold', 'color': color}),
            html.P(label, className="text-center mb-0", style={'color': '#666'})
        ])
    ], className="mb-3", style={'height': '150px'})

In [12]:
def create_big_numbers_section(df):
    big_numbers = calculate_big_numbers(df)
    
    return dbc.Container([
        html.H2("📊 Resumo Geral", className="mb-4"),
        dbc.Row([
            dbc.Col([
                create_number_card("✈️", f"{big_numbers['total_flights']:,}", "Total de Voos", "#3498db")
            ], width=6),
            dbc.Col([
                create_number_card("⏱️", f"{big_numbers['avg_delay']:.1f} min", "Atraso Médio", "#e74c3c")
            ], width=6),
        ], className="mb-3"),
        dbc.Row([
            dbc.Col([
                create_number_card("⚠️", f"{big_numbers['delay_percentage']:.1f}%", "Voos com Atraso", "#f39c12")
            ], width=4),
            dbc.Col([
                create_number_card("❌", f"{big_numbers['cancelled_percentage']:.1f}%", "Voos Cancelados", "#e67e22")
            ], width=4),
            dbc.Col([
                create_number_card("🔄", f"{big_numbers['diverted_percentage']:.1f}%", "Voos Desviados", "#9b59b6")
            ], width=4),
        ])
    ], fluid=True)

#### Aplicação Dash

In [13]:
# Inicializar a aplicação Dash com Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout da aplicação
app.layout = dbc.Container([
    html.Div([
        html.H1("✈️ Dashboard de Análise de Voos", 
                className="text-center mb-4",
                style={'background': 'linear-gradient(135deg, #667eea 0%, #764ba2 100%)',
                       'backgroundClip': 'text', 
                       'WebkitBackgroundClip': 'text',
                       'color': 'transparent',
                       'fontSize': '2.5em',
                       'fontWeight': 'bold'})
    ]),
    
    create_big_numbers_section(df),
    
    html.Hr(),
    
    html.H2("📈 Análise de Distribuições", className="mb-4"),
    
    html.Div([
        html.Label("Selecione a métrica para análise:", className="mb-3", style={'fontSize': '1.2em'}),
        dbc.RadioItems(
            id='metric-selector',
            options=[
                {'label': '⏱️ Média de Atraso', 'value': 'avg_delay'},
                {'label': '🔢 Quantidade de Atrasos', 'value': 'delay_count'},
                {'label': '❌ Quantidade de Cancelamentos', 'value': 'cancelled_count'},
                {'label': '🔄 Quantidade de Desvios', 'value': 'diverted_count'}
            ],
            value='avg_delay',
            inline=True,
            className="mb-4"
        )
    ]),
    
    # Gráficos organizados em grid
    dbc.Row([
        dbc.Col([dcc.Graph(id='airlines-chart')], width=6),
        dbc.Col([dcc.Graph(id='distance-chart')], width=6),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col([dcc.Graph(id='cities-chart')], width=6),
        dbc.Col([dcc.Graph(id='states-chart')], width=6),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col([dcc.Graph(id='day-chart')], width=6),
        dbc.Col([dcc.Graph(id='weekday-chart')], width=6),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col([dcc.Graph(id='hour-chart')], width=6),
        dbc.Col([dcc.Graph(id='period-chart')], width=6),
    ], className="mb-4"),
    
    html.Hr(),
    
    html.H2("🗺️ Visualização Geográfica", className="mb-4"),
    
    # Filtro para o mapa
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Label("Quantidade de rotas:", 
                        style={
                            'font-weight': 'bold', 
                            'margin-right': '15px',
                            'min-width': '150px',
                            'display': 'inline-block',
                            'vertical-align': 'middle'
                        }),
                html.Div(
                    dcc.Slider(
                        id='map-quantity-filter',
                        min=5,
                        max=100,
                        step=5,
                        value=30,
                        marks={
                            5: '5',
                            10: '10',
                            20: '20',
                            30: '30',
                            40: '40',
                            50: '50',
                            75: '75',
                            100: '100'
                        },
                        className="mb-3"
                    ),
                    style={'display': 'inline-block', 'width': 'calc(100% - 165px)', 'vertical-align': 'middle'}
                )
            ], style={'display': 'flex', 'align-items': 'center', 'width': '100%'})
        ], width=12)
    ]),
    
    dcc.Graph(id='map-chart'),
    
    # Footer
    html.Hr(),
    dbc.Container([
        html.Div([
            html.P([
                "Desenvolvido por ",
                html.Strong("Ianna Castro e Vitória Pistori"),
                " | Acesse o ",
                html.A("repositório original", 
                      href="https://github.com/vitoriapguimaraes/Python-AnaliseVoos",
                      target="_blank",
                      style={'color': '#3498db', 'textDecoration': 'none'}),
                " no GitHub"
            ], className="text-center mb-0", style={'color': '#666', 'fontSize': '0.9em'})
        ], style={'padding': '20px 0', 'borderTop': '1px solid #eee'})
    ], fluid=True)
    
], fluid=True, style={'padding': '20px'})

#### Callbacks

In [14]:
@app.callback(
    [Output('airlines-chart', 'figure'),
     Output('cities-chart', 'figure'),
     Output('states-chart', 'figure'),
     Output('distance-chart', 'figure'),
     Output('day-chart', 'figure'),
     Output('weekday-chart', 'figure'),
     Output('hour-chart', 'figure'),
     Output('period-chart', 'figure'),
     Output('map-chart', 'figure')],
    [Input('metric-selector', 'value'),
     Input('map-quantity-filter', 'value')]
)
def update_charts(selected_metric, map_quantity):
    return update_charts(selected_metric, map_quantity)

In [15]:
def update_charts(selected_metric, map_quantity):
    """Atualiza todos os gráficos baseado na métrica selecionada"""
    
    if not selected_metric:
        selected_metric = 'avg_delay'
    if not map_quantity:
        map_quantity = 10
    
    try:
        # Top 10 Companhias
        airlines_data, title_suffix = create_metric_data(df, "AIRLINE_Description", selected_metric)
        airlines_fig = create_simple_bar_chart(
            airlines_data.head(10),
            f"🏢 Top 10 Companhias - {title_suffix}", 
            title_suffix, 
            "Companhia"
        )
        
        # Distância
        df_temp = df.copy()
        df_temp["DISTANCE_BIN"] = pd.cut(df_temp["DISTANCE"], bins=10, precision=0)
        distance_data, _ = create_metric_data(df_temp, "DISTANCE_BIN", selected_metric, observed=True)
        distance_fig = create_simple_bar_chart(
            distance_data,
            f"✈️ Distância vs {title_suffix}", 
            title_suffix, 
            "Faixa de Distância"
        )
        
        # Top 10 Cidades
        cities_data, _ = create_metric_data(df, "ORIGIN_CITY", selected_metric)
        cities_fig = create_simple_bar_chart(
            cities_data.head(10),
            f"🏙️ Top 10 Cidades de Origem - {title_suffix}", 
            title_suffix, 
            "Cidade"
        )
        
        # Top 10 Estados
        states_data, _ = create_metric_data(df, "ORIGIN_STATE", selected_metric)
        states_fig = create_simple_bar_chart(
            states_data.head(10),
            f"🗺️ Top 10 Estados de Origem - {title_suffix}", 
            title_suffix, 
            "Estado"
        )
        
        # Dia do Mês
        day_data, _ = create_metric_data(df, "FL_DAY", selected_metric, observed=True)
        day_fig = create_line_chart_continuous(
            day_data,
            f"📅 Dia do Mês vs {title_suffix}", 
            "Dia do Mês", 
            title_suffix,
            "FL_DAY"
        )
        
        # Dia da Semana
        weekday_data, _ = create_metric_data(df, "DAY_OF_WEEK", selected_metric, observed=True)
        weekday_fig = create_simple_bar_chart(
            weekday_data,
            f"📆 Dia da Semana vs {title_suffix}", 
            title_suffix, 
            "Dia da Semana"
        )
        
        # Hora do Dia
        hour_data, _ = create_metric_data(df, "TIME_HOUR", selected_metric, observed=True)
        hour_fig = create_line_chart_continuous(
            hour_data,
            f"🕐 Hora do Dia vs {title_suffix}", 
            "Hora", 
            title_suffix,
            "TIME_HOUR"
        )
        
        # Período do Dia
        period_data, _ = create_metric_data(df, "TIME_PERIOD", selected_metric, observed=True)
        period_fig = create_simple_bar_chart(
            period_data,
            f"🌅 Período do Dia vs {title_suffix}", 
            title_suffix, 
            "Período"
        )
        
        # Mapa
        map_fig = criar_mapa_rotas_avancado(df, top_n=map_quantity, altura=600, 
                                          selected_metric=selected_metric)
        
        return (airlines_fig, cities_fig, states_fig, distance_fig, 
                day_fig, weekday_fig, hour_fig, period_fig, map_fig)
                
    except Exception as e:
        print(f"Erro ao criar gráficos: {e}")
        error_fig = px.bar(title=f"Erro: {str(e)}")
        error_fig.update_layout(height=400, title_x=0.5)
        return tuple([error_fig] * 9)

## Dashboard

In [16]:
if __name__ == '__main__':
    # Para Jupyter Notebook, use mode='inline'
    print("🚀 Iniciando Dashboard...")
    print("📊 Dados carregados com sucesso!")
    print("🌐 Acesse: http://127.0.0.1:8051")
    
    # Executar o dashboard
    app.run(debug=True, mode='inline', port=8051, height=800)

🚀 Iniciando Dashboard...
📊 Dados carregados com sucesso!
🌐 Acesse: http://127.0.0.1:8051
